In [2]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import torch
import pywt
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
def df_from_csv(file_path):
    # Read the csv file
    df_in = pd.read_csv(file_path)

    # Get the header row
    header_row = list(df_in.columns)

    # Convert the header row to integer values
    header_row = list(map(int, header_row))

    # Create a DataFrame from the header row
    header_df = pd.DataFrame([header_row], columns=df_in.columns)

    # Concatenate the header row with the original DataFrame
    df_imm = pd.concat([header_df, df_in], ignore_index=True)

    # Reset column names
    df_imm.columns = range(df_imm.shape[1])

    # Remove the first column
    df_imm = df_imm.drop(0, axis=1)

    df_out = df_imm.T
    
    return df_out

In [6]:
def z_score(epoch):    
    # Apply z-score normalization to each channel, saved in epoch
    for i in range(epoch.shape[0]):
        channel_epoch = epoch.iloc[i]
        mean = np.mean(channel_epoch)
        std = np.std(channel_epoch)
        z_scored_epoch = (channel_epoch - mean) / std
        epoch.iloc[i] = z_scored_epoch
    
    return epoch

def frequency_to_scale(freq, wavelet='morl', sampling_rate=250):
    # For the Morlet wavelet, scales are inversely proportional to frequency
    center_freq = pywt.central_frequency(wavelet)
    return center_freq / (freq / sampling_rate)

def apply_wavelet_transform(data_norm, wavelet='morl', freq_range=(8, 30), sampling_rate=250):
    """
    Apply wavelet transform to EEG data.
    
    Parameters:
    data_norm (ndarray): 2D array with shape (channels, time_points)
    wavelet (str): Wavelet type (default 'morl')
    freq_range (tuple): Frequency range for the CWT (default (8, 30) Hz)
    sampling_rate (int): Sampling rate of the EEG data (default 250 Hz)
    
    Returns:
    ndarray: 3D array with shape (channels, scales, time_points)
    """
    n_channels, n_times = data_norm.shape
    # Define scales based on the desired frequency range
    scales = frequency_to_scale(np.arange(freq_range[0], freq_range[1]+1), wavelet=wavelet, sampling_rate=sampling_rate)
    
    coeffs = []
    for i in range(n_channels):
        # Compute the wavelet transform coefficients
        coef, _ = pywt.cwt(data_norm.iloc[i], scales=scales, wavelet=wavelet)
        coeffs.append(coef)
    
    # Stack coefficients to form a 3D tensor
    coeffs_done = np.stack(coeffs, axis=0)
    
    return coeffs_done

def apply_bandpass_filter(signal, b, a):
    return filtfilt(b, a, signal)

def filtering_raw_signal(file_path):
    # Load the data, in dataframe format
    data_raw = df_from_csv(file_path)
    
    # Filter parameters
    l_freq = 8
    h_freq = 30
    order = 5
    fs = 250

    nyquist = 0.5 * fs
    low = l_freq / nyquist
    high = h_freq / nyquist

    # Calculate filter coefficients
    b, a = butter(order, [low, high], btype='band')
    
    # Apply the filter to each row independently
    for i in range(data_raw.shape[0]):
        data_raw.iloc[i] = apply_bandpass_filter(data_raw.iloc[i], b, a)
        
    # Convert filtered data back to DataFrame
    data_filtered = pd.DataFrame(data_raw)
    
    # Split data into segments of 0.5 seconds, for epoching
    epoch_size = int(fs * 0.5)  # Number of samples in 0.5 seconds
    epochs = [data_filtered.iloc[:, i:i+epoch_size] for i in range(0, len(data_filtered.columns), epoch_size)]
    
    # Create an empty list to store the transformed epochs
    transformed_epochs = []
    
    # Z-score normalization and wavelet transform for each segment and stack them into a tensor (4D array), 
    # Loop through each epoch
    for epoch in epochs:
        # Z-score each epoch
        epoch_norm = z_score(epoch)
        
        # # Apply wavelet transformation
        wavelet_tensor = apply_wavelet_transform(epoch_norm)
        
        # Append the transformed epoch to the list
        transformed_epochs.append(wavelet_tensor)
        
        
    # Convert the list of epochs into a tensor dataset
    tensor_dataset = torch.tensor(transformed_epochs, dtype=torch.float32)  # Ensure the tensor is float32
     
    return tensor_dataset

In [7]:
class Advanced_EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN, self).__init__()
        # First convolutional layer with Batch Normalization and ReLU activation
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Second convolutional layer with Batch Normalization and ReLU activation
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Third convolutional layer with Batch Normalization and ReLU activation
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 2 * 15, 256)  # Adjust based on final output size after conv layers
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        # Forward pass through the first convolutional layer
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Forward pass through the second convolutional layer
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Forward pass through the third convolutional layer
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        # Forward pass through the convolutional layers
        x = self._forward_conv_layers(x)
        # Flatten the tensor to feed into fully connected layers
        x = x.view(x.size(0), -1)
        # Forward pass through the first fully connected layer with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Forward pass through the second fully connected layer with ReLU
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x

In [3]:
class Advanced_EEG_MI_CNN_LSTM_Transformer(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN_LSTM_Transformer, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=128*2*15, hidden_size=128, num_layers=1, batch_first=True)
        
        # Transformer layer
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def _forward_conv_layers(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        batch_size = x.size(0)
        # Forward pass through convolutional layers
        x = self._forward_conv_layers(x)
        
        # Flatten for LSTM
        x = x.view(batch_size, -1, 128*2*15)
        
        # Forward pass through LSTM
        x, (hn, cn) = self.lstm(x)
        
        # Transformer encoding
        x = self.transformer_encoder(x)
        
        # Take the output from the last sequence step for classification
        x = x[:, -1, :]
        
        # Forward pass through fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:
def predict_cnn(model_path, data_to_predict):    
    # Load the state dictionary
    state_dict = torch.load(model_path)

    # Initialize the model and load the state dictionary
    model = Advanced_EEG_MI_CNN_LSTM_Transformer()
    model.load_state_dict(state_dict)
    model.eval()

    # Create TensorDataset and DataLoader
    tensor_dataset = filtering_raw_signal(data_to_predict)
    dataset = TensorDataset(tensor_dataset)
    dataloader = DataLoader(dataset, batch_size=10, shuffle=False)

    # Classify each sample
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            # Assuming your data is in the first element of the batch
            inputs = batch[0]
            
            # Move inputs to the same device as the model
            inputs = inputs.to(next(model.parameters()).device)
            
            # Get model predictions
            outputs = model(inputs)
            
            # Assuming the model output is logits, apply softmax to get probabilities
            probs = torch.softmax(outputs, dim=1)
            
            # Get the predicted class
            _, preds = torch.max(probs, 1)
            
            # Store predictions
            predictions.extend(preds.cpu().numpy())

    # Convert predictions to numpy array
    predictions = np.array(predictions)
    
    return predictions

NEWWWWW

In [15]:
# Load tensor dataset
tensor_dataset = torch.load(r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\openvibe\interim\openvibe_dataset_2024-06-08_21-04-15.pt")

# Labels are the last column of the tensor dataset
labels = tensor_dataset[:, :, :, -1]


# # Shuffle the combined dataset
# shuffle = True  # Set to True to shuffle the dataset
# batch_size = 8  # Set batch size as needed
# combined_dataloader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=shuffle)

IndexError: too many indices for tensor of dimension 1

AttributeError: 'TensorDataset' object has no attribute 'shape'

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.transforms import ToTensor

# Initialize your model
model = Advanced_EEG_MI_CNN_LSTM_Transformer()

# Load the trained model weights (assuming they are saved as a .pth file)
file = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\models\Advanced_EEG_MI_CNN_LSTM_Transformer_2024-06-08_16-01-15.pth"
model.load_state_dict(torch.load(file))

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

# Define a function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total
    return accuracy

# Iterate over the combined dataset
total_accuracy = 0.0
total_samples = 0
with torch.no_grad():
    for data, labels in combined_dataloader:  # Assuming combined_dataloader is your DataLoader for combined_dataset
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        batch_accuracy = calculate_accuracy(outputs, labels)
        total_accuracy += batch_accuracy * labels.size(0)
        total_samples += labels.size(0)

# Calculate overall accuracy
accuracy = total_accuracy / total_samples
print("Overall accuracy:", accuracy)

c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


ValueError: too many values to unpack (expected 2)